In [1]:
import pandas as pd
from IPython.display import display
import skipthoughts
import numpy as np
import time, datetime
import math
import keras
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout
from sklearn.metrics import accuracy_score as accuracy
import gc
tf.set_random_seed(5)
np.random.seed(5)

WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


[nltk_data] Downloading package punkt to /home/ubuntu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


/home/ubuntu/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# Provided data
train = pd.read_csv('data/train_stories.csv')
val = pd.read_csv('data/cloze_test_val__spring2016 - cloze_test_ALL_val.csv')
test = pd.read_csv('data/test_for_report-stories_labels.csv')

train = train.drop("storytitle", axis=1).drop("storyid", axis=1)

val = val.drop("InputStoryid", axis=1)
val_answer = val["AnswerRightEnding"]
val_sentences = val.drop("AnswerRightEnding", axis=1)

test = test.drop("InputStoryid", axis=1)
test_answer = test["AnswerRightEnding"]
test_sentences = test.drop("AnswerRightEnding", axis=1)

display(train.head())

display(val_sentences.head())
display(val_answer.head())

display(test_sentences.head())
display(test_answer.head())

,sentence1,sentence2,sentence3,sentence4,sentence5
0,Kelly found her grandmother's pizza recipe in ...,Kelly reminisced about how much she loved her ...,Kelly decided that she was going to try to mak...,Kelly studied the recipe and gathered everythi...,Kelly successfully made a pizza from her grand...
1,Leo wore a toupee and was anxious about it.,He decided to go out for a short walk.,"It was a very windy day, but he wasn't too con...","Suddenly, a strong wind came through and took ...","His dog leaped and caught it, and he quickly r..."
2,Jimmy was a master with his grill.,He spent every weekend getting better with his...,One day he was offered a TV show about grillin...,Jimmy accepted the job in an instant.,He quit his day job and spent all his time gri...
3,Tom and Beth were dating.,Tom wanted to go on a fun date.,Tom suggested ice skating.,His date was excited and they went ice skating.,Tom and Beth grew closer during their ice skat...
4,I had a friend that I didn't know well but let...,She paid rent then asked for some of it back.,She drinks my juice and eats my food.,She also makes a huge mess and is very sloppy.,She got kicked out in two weeks.


,InputSentence1,InputSentence2,InputSentence3,InputSentence4,RandomFifthSentenceQuiz1,RandomFifthSentenceQuiz2
0,Rick grew up in a troubled household.,"He never found good support in family, and tur...",It wasn't long before Rick got shot in a robbery.,The incident caused him to turn a new leaf.,He is happy now.,He joined a gang.
1,Laverne needs to prepare something for her fri...,She decides to bake a batch of brownies.,She chooses a recipe and follows it closely.,Laverne tests one of the brownies to make sure...,The brownies are so delicious Laverne eats two...,Laverne doesn't go to her friend's party.
2,Sarah had been dreaming of visiting Europe for...,She had finally saved enough for the trip.,She landed in Spain and traveled east across t...,She didn't like how different everything was.,Sarah then decided to move to Europe.,Sarah decided that she preferred her home over...
3,Gina was worried the cookie dough in the tube ...,She was very happy to find she was wrong.,The cookies from the tube were as good as from...,Gina intended to only eat 2 cookies and save t...,Gina liked the cookies so much she ate them al...,Gina gave the cookies away at her church.
4,It was my final performance in marching band.,I was playing the snare drum in the band.,We played Thriller and Radar Love.,The performance was flawless.,I was very proud of my performance.,I was very ashamed of my performance.


0    1
1    1
2    2
3    1
4    1
Name: AnswerRightEnding, dtype: int64

,InputSentence1,InputSentence2,InputSentence3,InputSentence4,RandomFifthSentenceQuiz1,RandomFifthSentenceQuiz2
0,My friends all love to go to the club to dance.,They think it's a lot of fun and always invite.,I finally decided to tag along last Saturday.,I danced terribly and broke a friend's toe.,My friends decided to keep inviting me out as ...,"The next weekend, I was asked to please stay h..."
1,I tried going to the park the other day.,The weather seemed nice enough for a walk.,Within minutes of getting there I started snee...,My eyes were watery and it was hard to breathe.,My allergies were too bad and I had to go back...,It reminded me of how much I loved spring flow...
2,Avery was married with children.,She was tired of her boring life.,"One day, she decided to meet up with an old bo...",She made poor decisions that night and was unf...,Avery thought her children would be happy with...,Avery regretted what she did the next day.
3,Josh loved when his mom baked apple pie.,He hated how he always had to wait until after...,So he decided this time he would sneak a piece...,The eggs his mom used must have been bad though.,Josh thought that the pie was delicious.,Josh got sick.
4,John was writing lyrics for his new album.,He started experiencing writer's block.,He tried to force himself to write but it woul...,"He took a walk, hung out with some friends, an...",He felt inspiration and then went back home to...,John then got an idea for his painting.


0    2
1    1
2    2
3    2
4    1
Name: AnswerRightEnding, dtype: int64

## Skipthoughts model from: https://github.com/ryankiros/skip-thoughts

In [3]:
a = time.time()
print("Loading model ...")
model = skipthoughts.load_model()
encoder = skipthoughts.Encoder(model)
print("Done in {} s".format(time.time() - a))

Loading model ...
Loading model parameters...
Compiling encoders...
Loading tables...
Packing up...
Done in 41.39524960517883 s


In [4]:
# a = time.time()
# print("Reading stories to memory ...")
# stories = []
# stories_flat = []
# for index, row in train.iterrows():
#     story = []
#     for col in train.columns:
#         story.append(row[col])
#         stories_flat.append(row[col])
#     stories.append(story)
# print("Done in {} s".format(time.time() - a))

In [5]:
# print("Nb of stories:", len(stories))
# print("Example story:", stories[0])
# print("Nb of sentences per story:", len(stories[0]))

In [6]:
# # testing skipthoughts encoder
# a = time.time()
# print("Encoding story ...")
# vectors = encoder.encode(stories[0], verbose=False)
# print("Done in {} s".format(time.time() - a))
# print("Encoded story:\n", vectors)
# print(type(vectors))
# print(vectors.shape)

In [7]:
# # encoding only some sentences of the story to see if the same encoding outputted
# a = time.time()
# print("Encoding part of story ...")
# vectors = encoder.encode(stories[0][0:3], verbose=False)
# print("Done in {} s".format(time.time() - a))
# print("Encoded story:\n", vectors)
# print(type(vectors))
# print(vectors.shape)

In [8]:
# encoded_sentences = np.zeros([len(stories_flat), 4800]) # each story having 5 sentences, each sentence encoded to a 
#                                                     # 4800-dim vector

# print(encoded_sentences)
# encoding_batch_size = 2000
# nb_batches = int(math.ceil(len(stories_flat) / encoding_batch_size))
# print("nb sentences per batch: {}, nb batches: {}".format(encoding_batch_size, nb_batches))
# for i in range(nb_batches):
#     a = time.time()
#     print("Encoding batch {} of sentences ...".format(i))
#     encoded_sentences[i*encoding_batch_size : (i+1)*encoding_batch_size] = encoder.encode(stories_flat[i*encoding_batch_size : (i+1)*encoding_batch_size], verbose=False)
#     print("Done in {} s".format(time.time() - a))
# print(encoded_sentences)

In [9]:
# encoded_stories = encoded_sentences.reshape([-1, 5, 4800])
# print(encoded_stories.shape)

In [10]:
def timestamp():
    return datetime.datetime.fromtimestamp(time.time()).strftime("%Y.%m.%d-%H:%M:%S")

def get_stories_as_lists(dataframe):
    a = time.time()
    print("Reading stories to memory ...")
    stories = []
    stories_flat = []
    for index, row in dataframe.iterrows():
        story = []
        for col in dataframe.columns:
            story.append(row[col])
            stories_flat.append(row[col])
        stories.append(story)
    print("Done in {} s".format(time.time() - a))
    return stories, stories_flat

def encode_stories(encoder, stories_flat, encoding_batch_size=2000):
    encoded_sentences = np.zeros([len(stories_flat), 4800]) # each sentence encoded to a 4800-dim vector

    nb_batches = int(math.ceil(len(stories_flat) / encoding_batch_size))
    print("nb sentences per batch: {}, nb batches: {}".format(encoding_batch_size, nb_batches))
    for i in range(nb_batches):
        a = time.time()
        print("Encoding batch {} of sentences ...".format(i))
        encoded_sentences[i*encoding_batch_size : (i+1)*encoding_batch_size] = encoder.encode(stories_flat[i*encoding_batch_size : (i+1)*encoding_batch_size], verbose=False)
        print("Done in {} s".format(time.time() - a))
    return encoded_sentences.reshape([-1, 6, 4800])

def get_train_valid_split(x_data, y_data, valid_percent=0.1, shuffle=True):
    data_size = x_data.shape[0]
    if shuffle:
        shuffle_indices = np.random.permutation(np.arange(data_size))
        x_data = x_data[shuffle_indices]
        y_data = y_data[shuffle_indices]
    split_index = int((1-valid_percent)*data_size)
    return x_data[:split_index], x_data[split_index:], y_data[:split_index], y_data[split_index:]

def split_pos_neg_endings(encoded_stories):
    encoded_stories_context = encoded_stories[:, 0:4] # first 4 sentences (i,e the context) 
    print(encoded_stories_context.shape)
    encoded_stories_context = np.repeat(encoded_stories_context, 2, axis=0) # repeat each context twice (once per possible ending)
    print(encoded_stories_context.shape)
    encoded_stories_endings = encoded_stories[:, 4:6] # the 2 possible endings
    print(encoded_stories_endings.shape)
    encoded_stories_endings = encoded_stories_endings.reshape([-1, 1, 4800]) # one ending per row
    print(encoded_stories_endings.shape)
    encoded_stories_split = np.concatenate([encoded_stories_context, encoded_stories_endings], axis=1)
    print(encoded_stories_split.shape)
    return encoded_stories_split

def create_stories_labels(answers):
    answers_split = np.zeros([2*answers.shape[0], 1])
    print("Answers shape:", answers_split.shape)
    for i,answer in enumerate(answers):
        if answer == 1:
            answers_split[2*i] = 1.
            answers_split[2*i+1] = 0.
        else: # answer == 2
            answers_split[2*i] = 0.
            answers_split[2*i+1] = 1.
    return answers_split

In [11]:
# --------------------------------------------- Labeled Train Data -----------------------------------------------

In [12]:
val_stories, val_stories_flat = get_stories_as_lists(val_sentences)

Reading stories to memory ...
Done in 0.20802617073059082 s


In [13]:
print("Nb of stories:", len(val_stories))
print("Example story with right and wrong ending:", val_stories[0])
print("Nb of sentences per story:", len(val_stories[0]))

Nb of stories: 1871
Example story with right and wrong ending: ['Rick grew up in a troubled household.', 'He never found good support in family, and turned to gangs.', "It wasn't long before Rick got shot in a robbery.", 'The incident caused him to turn a new leaf.', 'He is happy now.', 'He joined a gang.']
Nb of sentences per story: 6


In [14]:
val_encoded_stories = encode_stories(encoder, val_stories_flat)
print(val_encoded_stories.shape)
print(val_encoded_stories[0]) # print story 0 (both endings in the 5th and 6th sentence)

nb sentences per batch: 2000, nb batches: 6
Encoding batch 0 of sentences ...
Done in 22.15212845802307 s
Encoding batch 1 of sentences ...
Done in 22.93210196495056 s
Encoding batch 2 of sentences ...
Done in 22.670289516448975 s
Encoding batch 3 of sentences ...
Done in 23.3375985622406 s
Encoding batch 4 of sentences ...
Done in 22.92392110824585 s
Encoding batch 5 of sentences ...
Done in 15.111225366592407 s
(1871, 6, 4800)
[[ 0.00013156 -0.00460475  0.00544821 ... -0.01273214  0.00150634
  -0.00127442]
 [-0.00038869 -0.03386274 -0.00209803 ... -0.02370408  0.00628775
   0.00622181]
 [ 0.00784618  0.02995787 -0.00370612 ... -0.01775567  0.00297994
   0.01153545]
 [-0.00213756 -0.00957396 -0.00261727 ... -0.01276501  0.00276876
  -0.00384201]
 [-0.00714206 -0.0227032   0.01572856 ... -0.00376865  0.00276918
   0.00608158]
 [ 0.00167412 -0.01099871 -0.00196931 ... -0.03177596  0.0022835
   0.00621052]]


In [15]:
res = get_train_valid_split(val_encoded_stories, np.array(val_answer), valid_percent=0.1, shuffle=True)
train_stories, valid_stories, train_answers, valid_answers = res
print(train_stories.shape)
print(train_answers.shape)
print(valid_stories.shape)
print(valid_answers.shape)
print(train_stories[0]) # print train story 0 (both endings in the 5th and 6th sentence)

(1683, 6, 4800)
(1683,)
(188, 6, 4800)
(188,)
[[ 0.00257789 -0.01078219 -0.00668491 ... -0.03345989  0.00243997
   0.00695168]
 [ 0.00311376 -0.02050668  0.01805543 ... -0.00992052  0.00328235
   0.00122506]
 [ 0.00829043 -0.01595514 -0.00169544 ... -0.00554999  0.00448585
  -0.00629614]
 [ 0.01419469 -0.02284871 -0.00301768 ...  0.00108492  0.00252766
  -0.005558  ]
 [ 0.00201218 -0.02382531  0.01359426 ... -0.01885595  0.00522397
   0.00062915]
 [ 0.00917428 -0.00796457  0.00079475 ... -0.03304717  0.00208425
   0.00075519]]


In [16]:
train_stories_split = split_pos_neg_endings(train_stories)
print(train_stories_split[0:2]) # print same story as before with right and wrong endings on different rows
train_answers_split = create_stories_labels(train_answers) # 0 means the corresponding story in train_stories_split 
                                                      # has a wrong ending, and 1 means it has a right ending
print(train_answers_split[0:10])

(1683, 4, 4800)
(3366, 4, 4800)
(1683, 2, 4800)
(3366, 1, 4800)
(3366, 5, 4800)
[[[ 0.00257789 -0.01078219 -0.00668491 ... -0.03345989  0.00243997
    0.00695168]
  [ 0.00311376 -0.02050668  0.01805543 ... -0.00992052  0.00328235
    0.00122506]
  [ 0.00829043 -0.01595514 -0.00169544 ... -0.00554999  0.00448585
   -0.00629614]
  [ 0.01419469 -0.02284871 -0.00301768 ...  0.00108492  0.00252766
   -0.005558  ]
  [ 0.00201218 -0.02382531  0.01359426 ... -0.01885595  0.00522397
    0.00062915]]

 [[ 0.00257789 -0.01078219 -0.00668491 ... -0.03345989  0.00243997
    0.00695168]
  [ 0.00311376 -0.02050668  0.01805543 ... -0.00992052  0.00328235
    0.00122506]
  [ 0.00829043 -0.01595514 -0.00169544 ... -0.00554999  0.00448585
   -0.00629614]
  [ 0.01419469 -0.02284871 -0.00301768 ...  0.00108492  0.00252766
   -0.005558  ]
  [ 0.00917428 -0.00796457  0.00079475 ... -0.03304717  0.00208425
    0.00075519]]]
Answers shape: (3366, 1)
[[1.]
 [0.]
 [1.]
 [0.]
 [1.]
 [0.]
 [1.]
 [0.]
 [1.]
 [0.]]


In [17]:
valid_stories_split = split_pos_neg_endings(valid_stories)
valid_answers_split = create_stories_labels(valid_answers) # 0 means the corresponding story in train_stories_split 
                                                      # has a wrong ending, and 1 means it has a right ending

(188, 4, 4800)
(376, 4, 4800)
(188, 2, 4800)
(376, 1, 4800)
(376, 5, 4800)
Answers shape: (376, 1)


In [18]:
# ----------------------------------------------- Test Data ------------------------------------------------------

In [19]:
test_stories, test_stories_flat = get_stories_as_lists(test_sentences)

Reading stories to memory ...
Done in 0.22518086433410645 s


In [20]:
print("Nb of stories:", len(test_stories))
print("Example story with right and wrong ending:", test_stories[0])
print("Nb of sentences per story:", len(test_stories[0]))

Nb of stories: 1871
Example story with right and wrong ending: ['My friends all love to go to the club to dance.', "They think it's a lot of fun and always invite.", 'I finally decided to tag along last Saturday.', "I danced terribly and broke a friend's toe.", 'My friends decided to keep inviting me out as I am so much fun.', 'The next weekend, I was asked to please stay home.']
Nb of sentences per story: 6


In [21]:
test_encoded_stories = encode_stories(encoder, test_stories_flat)
print(test_encoded_stories.shape)
print(test_encoded_stories[0]) # print story 0 (both endings in the 5th and 6th sentence)

nb sentences per batch: 2000, nb batches: 6
Encoding batch 0 of sentences ...
Done in 24.234635829925537 s
Encoding batch 1 of sentences ...
Done in 23.174983263015747 s
Encoding batch 2 of sentences ...
Done in 22.74548888206482 s
Encoding batch 3 of sentences ...
Done in 22.02769923210144 s
Encoding batch 4 of sentences ...
Done in 22.95640277862549 s
Encoding batch 5 of sentences ...
Done in 14.577431678771973 s
(1871, 6, 4800)
[[-0.01622678 -0.01267685 -0.00449958 ... -0.01958963  0.0025401
   0.00582471]
 [ 0.00351208  0.01862131  0.01466644 ... -0.02046947  0.00296836
  -0.00029696]
 [-0.00341649 -0.02440572  0.02018603 ... -0.03569067  0.00320727
  -0.00057357]
 [ 0.02745583 -0.0154904  -0.00947571 ...  0.0108288   0.00337028
   0.01885112]
 [-0.00548592 -0.03024452  0.02510787 ... -0.00660004  0.00243402
  -0.00326103]
 [ 0.00078507 -0.02019084  0.01723271 ... -0.03500823  0.00463203
  -0.00075694]]


In [22]:
test_stories_split = split_pos_neg_endings(test_encoded_stories)
print(test_stories_split[0:2]) # print same story as before with right and wrong ending on different rows
test_answers = np.array(test_answer)

(1871, 4, 4800)
(3742, 4, 4800)
(1871, 2, 4800)
(3742, 1, 4800)
(3742, 5, 4800)
[[[-0.01622678 -0.01267685 -0.00449958 ... -0.01958963  0.0025401
    0.00582471]
  [ 0.00351208  0.01862131  0.01466644 ... -0.02046947  0.00296836
   -0.00029696]
  [-0.00341649 -0.02440572  0.02018603 ... -0.03569067  0.00320727
   -0.00057357]
  [ 0.02745583 -0.0154904  -0.00947571 ...  0.0108288   0.00337028
    0.01885112]
  [-0.00548592 -0.03024452  0.02510787 ... -0.00660004  0.00243402
   -0.00326103]]

 [[-0.01622678 -0.01267685 -0.00449958 ... -0.01958963  0.0025401
    0.00582471]
  [ 0.00351208  0.01862131  0.01466644 ... -0.02046947  0.00296836
   -0.00029696]
  [-0.00341649 -0.02440572  0.02018603 ... -0.03569067  0.00320727
   -0.00057357]
  [ 0.02745583 -0.0154904  -0.00947571 ...  0.0108288   0.00337028
    0.01885112]
  [ 0.00078507 -0.02019084  0.01723271 ... -0.03500823  0.00463203
   -0.00075694]]]


In [23]:
# ------------------------------------------ Model and Training -----------------------------------------------

In [24]:
# train: last sentence only + ending
train_stories_ls = train_stories_split[:, 3] + train_stories_split[:, 4]
print(train_stories_ls.shape)
train_answers_categorical = keras.utils.to_categorical(train_answers_split, num_classes=2, dtype='float32')
print(train_answers_split)
print(train_answers_categorical)

(3366, 4800)
[[1.]
 [0.]
 [1.]
 ...
 [1.]
 [1.]
 [0.]]
[[0. 1.]
 [1. 0.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [1. 0.]]


In [25]:
# valid: last sentence only + ending
valid_stories_ls = valid_stories_split[:, 3] + valid_stories_split[:, 4]
print(valid_stories_ls.shape)
valid_answers_categorical = keras.utils.to_categorical(valid_answers_split, num_classes=2, dtype='float32')

(376, 4800)


In [26]:
# test: last sentence only + ending
test_stories_ls = test_stories_split[:, 3] + test_stories_split[:, 4]
print(test_stories_ls.shape)

(3742, 4800)


In [27]:
def create_model():
    model = Sequential()
    model.add(Dense(units=2400, activation='relu', input_dim=4800))
    model.add(Dense(units=1200, activation='relu'))
    model.add(Dense(units=600, activation='relu'))
    model.add(Dense(units=2, activation='softmax')) # output layer
    return model

In [28]:
class WriterCallback(keras.callbacks.Callback):
    def __init__(self):
        self.writer = tf.summary.FileWriter('./logs/{}'.format(timestamp()))
        
        self.train_loss_pl = tf.placeholder(dtype=tf.float32)
        self.valid_loss_pl = tf.placeholder(dtype=tf.float32)
        self.valid_accuracy_pl = tf.placeholder(dtype=tf.float32)
        self.test_accuracy_pl = tf.placeholder(dtype=tf.float32)
        self.epoch_summary = tf.summary.merge(
            [tf.summary.scalar("train_loss", self.train_loss_pl), 
             tf.summary.scalar("valid_loss", self.valid_loss_pl),
             tf.summary.scalar("valid_accuracy", self.valid_accuracy_pl),
             tf.summary.scalar("test_accuracy", self.test_accuracy_pl)
            ])
    
    def on_epoch_end(self, epoch, logs={}):
        valid_pred = self.get_predicted_right_endings(valid_stories_ls)
        test_pred = self.get_predicted_right_endings(test_stories_ls)
        
        print("\n")
        print("logs:", logs)
        print("Validation:")
#         print("   Predicted endings:", valid_pred)
#         print("   Correct endings:", valid_answers)
        valid_acc = accuracy(valid_answers, valid_pred)
        print("   Validation Accuracy: {}".format(valid_acc))
        
        print("Test:")
#         print("   Predicted endings:", test_pred)
#         print("   Correct endings:", test_answers)
        test_acc = accuracy(test_answers, test_pred)
        print("   Test Accuracy: {}".format(test_acc))
        print("\n")
        train_loss = logs["loss"]
        valid_loss = logs["val_loss"]
        
        with tf.Session() as sess:
            summary = sess.run(self.epoch_summary, {self.train_loss_pl: train_loss,
                                          self.valid_loss_pl: valid_loss,
                                          self.valid_accuracy_pl: valid_acc,
                                          self.test_accuracy_pl: test_acc})
        self.writer.add_summary(summary, epoch)
        self.writer.flush()
        
    def on_train_end(self, logs=None):
        self.writer.flush()
        self.writer.close()
    
    def get_predicted_right_endings(self, data):
        preds = model.predict(data)
        preds_endings = []
        for i in range(preds.shape[0]//2):
            if(preds[2*i][1] > preds[2*i+1][1]): # if the first ending is "more right" then the second ending
                preds_endings.append(1)
            else:
                preds_endings.append(2)
        return np.array(preds_endings)

In [29]:
gc.collect()
# create and compile model
model = create_model()
model.compile(loss='categorical_crossentropy',
              optimizer=keras.optimizers.Adam(lr=0.001))
# define callbacks
writerCallback = WriterCallback()

# train model
model.fit(train_stories_ls, train_answers_categorical, epochs=100, batch_size=32, 
          callbacks=[writerCallback], validation_data=[valid_stories_ls, valid_answers_categorical])

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Train on 3366 samples, validate on 376 samples
Epoch 1/100
3366/3366 [==============================] - 4s 1ms/step - loss: 0.6960 - val_loss: 0.6838


logs: {'val_loss': 0.6837926608450869, 'loss': 0.6960098699169079}
Validation:
   Validation Accuracy: 0.6968085106382979
Test:
   Test Accuracy: 0.7113842864778194


Epoch 2/100
3366/3366 [==============================] - 2s 639us/step - loss: 0.6755 - val_loss: 0.6550


logs: {'val_loss': 0.6550100204792428, 'loss': 0.675471656639804}
Validation:
   Validation Accuracy: 0.7074468085106383
Test:
   Test Accuracy: 0.7151256012827365


Epoch 3/100
3366/3366 [==============================] - 2s 641us/step - loss: 0.6430 - val_loss: 0.6828


logs: {'val_loss': 0.6828215984587974, 'loss': 0.6430247247537827}
Validation:
   Validation Accuracy: 0.7127659574468085
Test:
   Test Accuracy: 0.7268840192410476


Epoch 4/100
33

3366/3366 [==============================] - 2s 647us/step - loss: 0.0279 - val_loss: 1.9889


logs: {'val_loss': 1.9888727563492796, 'loss': 0.0278949630823024}
Validation:
   Validation Accuracy: 0.7659574468085106
Test:
   Test Accuracy: 0.740780331373597


Epoch 30/100
3366/3366 [==============================] - 2s 645us/step - loss: 0.0389 - val_loss: 2.1360


logs: {'val_loss': 2.1359539666074387, 'loss': 0.03891806454860909}
Validation:
   Validation Accuracy: 0.7606382978723404
Test:
   Test Accuracy: 0.7386424371993586


Epoch 31/100
3366/3366 [==============================] - 2s 646us/step - loss: 0.0214 - val_loss: 2.1382


logs: {'val_loss': 2.138191010089631, 'loss': 0.021370711088107498}
Validation:
   Validation Accuracy: 0.7393617021276596
Test:
   Test Accuracy: 0.7338321753073223


Epoch 32/100
3366/3366 [==============================] - 2s 647us/step - loss: 0.0701 - val_loss: 1.7579


logs: {'val_loss': 1.757862410646804, 'loss': 0.07010384339593524}
Validation:


3366/3366 [==============================] - 2s 646us/step - loss: 0.0271 - val_loss: 2.1656


logs: {'val_loss': 2.1656397378191037, 'loss': 0.02707621420478942}
Validation:
   Validation Accuracy: 0.7393617021276596
Test:
   Test Accuracy: 0.7423837520042758


Epoch 60/100
3366/3366 [==============================] - 2s 647us/step - loss: 0.0051 - val_loss: 2.3454


logs: {'val_loss': 2.3453781579403166, 'loss': 0.0051128855656441775}
Validation:
   Validation Accuracy: 0.7553191489361702
Test:
   Test Accuracy: 0.7322287546766435


Epoch 61/100
3366/3366 [==============================] - 2s 647us/step - loss: 0.0145 - val_loss: 2.2664


logs: {'val_loss': 2.2663560040453645, 'loss': 0.014533739587763755}
Validation:
   Validation Accuracy: 0.7606382978723404
Test:
   Test Accuracy: 0.7354355959380011


Epoch 62/100
3366/3366 [==============================] - 2s 648us/step - loss: 0.0196 - val_loss: 2.2237


logs: {'val_loss': 2.2236639235882047, 'loss': 0.019574608807625123}
Valid

3366/3366 [==============================] - 2s 646us/step - loss: 4.7962e-04 - val_loss: 2.8441


logs: {'val_loss': 2.8441195792340217, 'loss': 0.0004796194425730286}
Validation:
   Validation Accuracy: 0.7659574468085106
Test:
   Test Accuracy: 0.7359700694815606


Epoch 90/100
3366/3366 [==============================] - 2s 646us/step - loss: 7.9126e-04 - val_loss: 2.8531


logs: {'val_loss': 2.8530865907669067, 'loss': 0.0007912611535604067}
Validation:
   Validation Accuracy: 0.7712765957446809
Test:
   Test Accuracy: 0.738107963655799


Epoch 91/100
3366/3366 [==============================] - 2s 646us/step - loss: 5.5674e-04 - val_loss: 2.8994


logs: {'val_loss': 2.899423084360488, 'loss': 0.0005567389385126797}
Validation:
   Validation Accuracy: 0.7659574468085106
Test:
   Test Accuracy: 0.7316942811330839


Epoch 92/100
3366/3366 [==============================] - 2s 647us/step - loss: 5.0487e-04 - val_loss: 2.9826


logs: {'val_loss': 2.9826174492531634, 'loss': 0.00050487